In [1]:
#Pip install selenium and webdriver_manager if not already done

import selenium
from selenium import webdriver
import time
import requests
import os
from PIL import Image
import io
import hashlib
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup

In [2]:
# Put the path for your ChromeDriver here
DRIVER_PATH = "ChromeDriverManager().install()"

In [3]:
def fetch_image_urls(query:str, no_of_pages_of_60:int, wd:webdriver, sleep_between_interactions:int=1):
    
    q = query
    
    wd.get(f"https://www.google.co.uk/search?q={q}&hl=en&psb=1&tbs=vw:d&tbm=shop&ei=cn_-Y9DnFt_87_UP2d6-6Ac&start=0&sa=N&ved=0ahUKEwjQ6Luhobn9AhVf_rsIHVmvD30Q8tMDCI4R&biw=1470&bih=733&dpr=2")
    wd.find_element(By.XPATH, "//button[contains(@class, 'VfPpkd')]").click()
    
    image_urls = []
    for n in range(no_of_pages_of_60):
        search_url =f"https://www.google.co.uk/search?q={q}&hl=en&psb=1&tbs=vw:d&tbm=shop&ei=cn_-Y9DnFt_87_UP2d6-6Ac&start={n*60}&sa=N&ved=0ahUKEwjQ6Luhobn9AhVf_rsIHVmvD30Q8tMDCI4R&biw=1470&bih=733&dpr=2"
        wd.get(search_url)

    #get all image thumbnail results
        print('Processing: ')
        for j in range(1,61):
            try:
                thumbnail_xpath = f"/html/body/div[7]/div/div[4]/div[3]/div/div[3]/div[2]/div[2]/div/div[{j}]/div[1]/div[2]/div[1]/div[1]/div/div/a/div/div/img"
                
                thumbnail_results = wd.find_element(By.XPATH, thumbnail_xpath).get_attribute('src')
                image_urls.append(thumbnail_results)
                print(f"{j}/60 ", end="")

            except Exception:
                continue
    
        image_count = len(image_urls)
        print(f"image count: {image_count} after page {n+1}")
        
    return set(image_urls)


In [4]:
def persist_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [5]:
#change your set of queries here, eg. carrots
query = "carrots"
#set number of pages to scrape -- google shopping search returns 60 items per page
no_of_pages_of_60_to_scrape = 2
#images_path_folder
images_path = '/Users/username/desktop/download_images'

wd = webdriver.Chrome(executable_path=DRIVER_PATH)
links = fetch_image_urls(query, no_of_pages_of_60_to_scrape ,wd)
    
for i in links:
    persist_image(images_path,query,i)

/var/folders/51/1bfgt4ls78d2ckf9flwx16jm0000gn/T/ipykernel_52151/3457071167.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=DRIVER_PATH)


Processing: 
1/60 2/60 3/60 4/60 5/60 6/60 7/60 8/60 9/60 10/60 11/60 12/60 13/60 14/60 15/60 16/60 17/60 18/60 19/60 20/60 21/60 22/60 23/60 24/60 25/60 26/60 27/60 28/60 29/60 30/60 31/60 32/60 33/60 34/60 35/60 36/60 37/60 38/60 39/60 40/60 41/60 42/60 43/60 44/60 45/60 46/60 47/60 48/60 49/60 50/60 51/60 52/60 53/60 54/60 55/60 56/60 57/60 58/60 59/60 60/60 image count: 60 after page 1
Processing: 
1/60 2/60 3/60 4/60 5/60 6/60 7/60 8/60 9/60 10/60 11/60 12/60 13/60 14/60 15/60 16/60 17/60 18/60 19/60 20/60 21/60 22/60 23/60 24/60 25/60 26/60 27/60 28/60 29/60 30/60 31/60 32/60 33/60 34/60 35/60 36/60 37/60 38/60 39/60 40/60 41/60 42/60 43/60 44/60 45/60 46/60 47/60 48/60 49/60 50/60 51/60 52/60 53/60 54/60 55/60 56/60 57/60 58/60 59/60 60/60 image count: 120 after page 2
SUCCESS - saved https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcSoLrXMS0heLcLbXgz3laJHX5Ys3jHecRDdLFm1s_lu3eYbeU4Jr6iNUFuYQEMDHLGKzRLk3-AFyAyCTM5qR6ZOTAALX4LmvsyRpDJaKhF2KE7eTCD6LWrO&usqp=CAE - as /Users/c

SUCCESS - saved https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcRnD0aik0zHSCjFNuDzZARG-Jl0a2hyk3ZOstT81McQHNFQxUMFLIHxAXht0P8lSoLrdW_Qv0FzKxf057qziNDlGjwSdRXX8cFSzVTyW0Ox&usqp=CAE - as /Users/candy/Desktop/DL/carrots/9d70b40beb.jpg
SUCCESS - saved https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcSdX3tR6COKWXynVG23loiqPSlAgbl6ZCRGMYrGhAv6akvs5p4KPbiI4gO6w9VdwylW_Wj8Xgmwa_fCMO6ndsNgE4A_llDgVzMHU-pKi7hm5tEz4FG9iNdg7g&usqp=CAE - as /Users/candy/Desktop/DL/carrots/598ac1c9bd.jpg
SUCCESS - saved https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcRDj7iqD6jtv_wrCH62jUlcxdZu1a6Go3DPUNYFECJDmYB9LVGzSjIIR4kauVTPrZ0_5PJteRD0sh6phiBdtrpmPv0HFBQajd4Wd_764fcufhqvwBXaM8Bq&usqp=CAE - as /Users/candy/Desktop/DL/carrots/98f331eee4.jpg
SUCCESS - saved https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcT0rpEIbY6xu1diYRWoa9FaIeooqiHTz3DD5VaqYoFiDQAqLv8ckFozqoy3QQt6b5PhFR1ixAh1ByFvYP2Yynbki6HJtEakJw&usqp=CAE - as /Users/candy/Desktop/DL/carrots/3fa88d580e.jpg
SUCCESS - saved https://

SUCCESS - saved https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcSXEAVhwugDON1CR3xVnEYKz1CfPNkGaADDT5mtxp8s6CYMzAxqJ2IACQg1AXa_cW_-QTY8Chq_y_jIuZifkn8ndtGVhPkQ&usqp=CAE - as /Users/candy/Desktop/DL/carrots/a64bac658e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcTltFaPc_VCoTf7onqkdGfxnjU8Bo-dGz52C--LNGAZUlSmbznXjQ6PoahZ-DKgTpwWyzIU_4hmWZUYfxYUafXp9fUOqAOVsN5MdQ14FAqNBe_HsqNfi2te&usqp=CAE - as /Users/candy/Desktop/DL/carrots/e4aed70726.jpg
SUCCESS - saved https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcQGnAE7O2q01Yr8OfL4SjiBBg089WxLVMn-paceqfroQ0F0r3J1lqu13nwtuNs-GgasDMJo5O9iAtFcl1SvtW3CXf-OsWs-&usqp=CAE - as /Users/candy/Desktop/DL/carrots/070d0ef507.jpg
SUCCESS - saved https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcS04cpFqv2kAUBEihP9eW27t6aw8QVkjGXmITvUJExhsAsWg3y1BIYFJNIrccsHfmZZcWo3elR0ETZpsVNSilrd3g1vFcramGmfvopS2Taza_HzxBZ5rqZ08g&usqp=CAE - as /Users/candy/Desktop/DL/carrots/288ea9ce19.jpg
SUCCESS - saved https://encrypted-tbn1

SUCCESS - saved https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcTWXGfksedUgJ_4HttolqcuTTkonh0YFeEowoZBOZvigxm3gpFXxoDkJsQ-6_Y69DUdAIfcHeVepG806akpJGDwEaIYq10a98EgtOFRJeouFHG0WSoO1lYHKA&usqp=CAE - as /Users/candy/Desktop/DL/carrots/b8d1958fc4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcTFYNXpfMfg1ed0OZPlkU_UQtR76Pj40ELhV8diaGa40LFSKxnKu8U-MSDjB1ULccW-DShJ-8JVdPPcJJ2QSAIXsA30bvWtTJZmZjAg3Yes&usqp=CAE - as /Users/candy/Desktop/DL/carrots/4fe5924ac3.jpg
SUCCESS - saved https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcTkdU9EHMdqHEp_VPZz75x7fhZBnfcx43rv0pX_wMu0IaPYOqB-D-eFFqViEl20SaSTJDjpLsIRpPmSP7rQJ2YkFXY1Azi7t3Y_zzQAH2T0YVlFZrQJntBAxA&usqp=CAE - as /Users/candy/Desktop/DL/carrots/4431ef7bb3.jpg
SUCCESS - saved https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcR_QrXGwFWZAFI1HGQgASULIr4PqLr-k0UEURHTOiBknSdlWdJcuUzRYZG1LeGbx1dGKxDa5_hfpPxRJr6hzKGA-aj-GRGTTnDgO9yBd-b7WnWyobO4ECsFo68&usqp=CAE - as /Users/candy/Desktop/DL/carrots/04beaeba63.j